# local run prepare

In [ ]:
pip install --user tensorflow-gpu==2.10.0 NumPy==1.23.5 scikit-image==0.19.3 protobuf==3.19.6 googleapis-common-protos==1.60.0 imbalanced-learn==0.11.0

In [ ]:
import os
import json
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi

import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import random
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPool2D, GlobalMaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import Model

import visualkeras

import lime
from lime import lime_image
from lime import submodular_pick

from skimage.segmentation import mark_boundaries

import matplotlib.pyplot as plt

import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from tensorflow.python.client import device_lib
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.mixed_precision import set_global_policy
from tensorflow.keras import backend as K

## GPU prepareing

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)

In [ ]:
print(device_lib.list_local_devices())

In [ ]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPU.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found.")

In [ ]:
set_global_policy('mixed_float16')

In [ ]:
#clean gpu memory
K.clear_session()

# Data prepareing

In [ ]:
import os
import json
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi

In [ ]:
print(os.listdir('./vehicle_data'))

# Experiments

## Unbalnce data

In [ ]:
import tensorflow.keras.backend as K

def initialize_environment(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

initialize_environment(seed=42)

def prepare_data_generators(dataset_path, img_size=(224, 224), batch_size=32, seed=42):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )

    train_data = train_datagen.flow_from_directory(
        dataset_path,
        target_size=img_size,
        batch_size=batch_size,
        subset='training',
        class_mode='categorical',
        seed=seed
    )

    validation_data = train_datagen.flow_from_directory(
        dataset_path,
        target_size=img_size,
        batch_size=batch_size,
        subset='validation',
        class_mode='categorical',
        shuffle=False,
        seed=seed
    )

    print("Class indices:", train_data.class_indices)
    print("Number of classes:", len(validation_data.class_indices))
    return train_data, validation_data

In [ ]:
train_data, validation_data = prepare_data_generators(dataset_path='./vehicle_data/Vehicle Type Image Dataset (Version 2) VTID2')
class_names = ['Hatchback', 'Other', 'Pickup', 'SUV', 'Seden']
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    images, labels = train_data[i]
    plt.imshow(images[i % images.shape[0]])
    plt.xlabel(class_names[np.argmax(labels[i % labels.shape[0]])])
plt.show()

### Baseline model testing (before image segmentation)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def base_model(input_shape=(224, 224, 3), num_classes=5):
    model = models.Sequential()

    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
def train_model(model, train_data, validation_data, epochs=10):
    history = model.fit(
        train_data,
        validation_data=validation_data,
        epochs=epochs,
    )
    return history

In [ ]:
def plot_training_metrics(history, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy"):
    plt.figure(figsize=(10, 10))

    # Plot Loss
    plt.subplot(2, 2, 1)
    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title(loss_title)

    # Plot Accuracy
    plt.subplot(2, 2, 2)
    plt.plot(history.history['accuracy'], label='Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title(accuracy_title)

    plt.show()


In [ ]:
def basic_model(seed=42, dataset_path='./vehicle_data/Vehicle Type Image Dataset (Version 2) VTID2'):

    initialize_environment(seed)

    train_data, validation_data = prepare_data_generators(dataset_path, seed=seed)

    model = base_model()

    history = train_model(model, train_data, validation_data, epochs=10)

    plot_training_metrics(history)

    K.clear_session()

    return model, train_data, validation_data

base_model_unbalanced, train_data, validation_data = basic_model(seed=42)

In [ ]:
visualkeras.layered_view(base_model_unbalanced, legend=True)

### duplicates search

In [ ]:
import os
import shutil
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.io import imread

def pairwise_duplicate_removal(input_folder, output_folder, ssim_threshold=0.9):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for class_folder in os.listdir(input_folder):
        class_path = os.path.join(input_folder, class_folder)
        if not os.path.isdir(class_path):
            continue

        print(f"Processing class: {class_folder}")
        output_class_folder = os.path.join(output_folder, class_folder)
        os.makedirs(output_class_folder, exist_ok=True)

        images = os.listdir(class_path)
        num_images = len(images)
        to_keep = set(range(num_images))

        for i in range(num_images):
            if i not in to_keep:
                continue

            img1_path = os.path.join(class_path, images[i])
            img1 = imread(img1_path, as_gray=True)

            for j in range(i + 1, num_images):
                if j not in to_keep:
                    continue

                img2_path = os.path.join(class_path, images[j])
                img2 = imread(img2_path, as_gray=True)

                if img1.shape != img2.shape:
                    min_shape = np.minimum(img1.shape, img2.shape)
                    img1 = img1[:min_shape[0], :min_shape[1]]
                    img2 = img2[:min_shape[0], :min_shape[1]]

                similarity = ssim(img1, img2)
                if similarity > ssim_threshold:
                    print(f"Duplicate found: {images[i]} and {images[j]} (SSIM: {similarity})")
                    to_keep.discard(j)

        for idx in to_keep:
            src_path = os.path.join(class_path, images[idx])
            dst_path = os.path.join(output_class_folder, images[idx])
            shutil.copy(src_path, dst_path)

    print("Duplicate removal complete.")


pairwise_duplicate_removal(
    input_folder='./vehicle_data/Vehicle Type Image Dataset (Version 2) VTID2',
    output_folder='./vehicle_data/Vehicle Type Image Dataset (Version 2) VTID2 cleaned',
    ssim_threshold=0.80
)

print("Duplicates found:")
for duplicate in duplicates:
    print(duplicate)

In [ ]:
def basic_model(seed=42, dataset_path='./vehicle_data/Vehicle Type Image Dataset (Version 2) VTID2 cleaned'):

    initialize_environment(seed)

    train_data, validation_data = prepare_data_generators(dataset_path, seed=seed)

    model = base_model()

    history = train_model(model, train_data, validation_data, epochs=10)

    plot_training_metrics(history)

    K.clear_session()

    return model, train_data, validation_data

base_model_unbalanced, train_data, validation_data = basic_model(seed=42)

In [ ]:
def lime_pipeline(model, image_paths, img_size=(224, 224), title="LIME Results", num_features=5, num_samples=5000):

    def preprocess_image(img_path):
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
        return np.expand_dims(img_array, axis=0), img_array

    explainer = lime_image.LimeImageExplainer()

    n_images = len(image_paths)
    cols = 5
    rows = (n_images + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(15, rows * 5))
    axes = axes.flatten()

    for idx, img_path in enumerate(image_paths):
        preprocessed_img, original_img = preprocess_image(img_path)

        prediction = model.predict(preprocessed_img)
        predicted_class = np.argmax(prediction, axis=1)[0]
        print(f"DEBUG: Prediction for image {idx+1}: {prediction}, Predicted class: {predicted_class}")  # Debugging line

        explanation = explainer.explain_instance(
            original_img,
            model.predict,
            top_labels=1,
            hide_color=0,
            num_samples=num_samples
        )

        temp, mask = explanation.get_image_and_mask(
            explanation.top_labels[0],
            positive_only=True,
            num_features=num_features,
            hide_rest=False
        )

        if idx < len(axes):
            axes[idx].imshow(mark_boundaries(temp, mask))
            axes[idx].axis('off')
            axes[idx].set_title(f"Predicted: Class {predicted_class}")

    for i in range(len(image_paths), len(axes)):
        axes[i].axis('off')

    fig.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
model = base_model_unbalanced
image_paths = ["./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Hatchback\PHOTO_14.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Other\PHOTO_11.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Pickup\PHOTO_3.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Seden\PHOTO_7.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\SUV\PHOTO_17.jpg"]
lime_pipeline(model, image_paths, title="Base Model (Data Cleaned) LIME Results")

## Balance data

In [ ]:
import os
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import Image
import numpy as np
import shutil
import Augmentor

In [ ]:
def augment_dataset(
    input_dataset_path,
    output_dataset_path,
    target_images_per_class=5000,
    augment_output_name="output_"
):

    print("Copying original images...")
    os.makedirs(output_dataset_path, exist_ok=True)
    for cls in os.listdir(input_dataset_path):
        class_dir = os.path.join(input_dataset_path, cls)
        augmented_class_dir = os.path.join(output_dataset_path, cls)

        os.makedirs(augmented_class_dir, exist_ok=True)

        for img_file in os.listdir(class_dir):
            src = os.path.join(class_dir, img_file)
            dst = os.path.join(augmented_class_dir, img_file)
            if not os.path.exists(dst):
                shutil.copy(src, dst)

    print("Original images copied successfully.")

    for cls in os.listdir(output_dataset_path):
        class_dir = os.path.join(output_dataset_path, cls)

        if not os.path.isdir(class_dir):
            continue

        num_images = len(os.listdir(class_dir))
        num_to_generate = target_images_per_class - num_images

        if num_to_generate <= 0:
            print(f"{cls} already has sufficient images ({num_images}). No augmentation needed.")
            continue

        print(f"Augmenting {cls}: Generating {num_to_generate} new images.")

        p = Augmentor.Pipeline(source_directory=class_dir, output_directory=".")

        # augmentation operations
        p.rotate(probability=0.7, max_left_rotation=15, max_right_rotation=15)
        p.flip_left_right(probability=0.5)
        p.zoom_random(probability=0.5, percentage_area=0.8)
        p.shear(probability=0.5, max_shear_left=10, max_shear_right=10)
        p.random_brightness(probability=0.5, min_factor=0.8, max_factor=1.2)
        p.random_distortion(probability=0.8, grid_width=4, grid_height=4, magnitude=8)
        p.sample(num_to_generate)

        for img_file in os.listdir("."):
            if img_file.startswith(augment_output_name) and img_file.endswith((".jpg", ".png")):
                shutil.move(img_file, class_dir)

        print(f"Finished augmenting {cls}. Total images: {len(os.listdir(class_dir))}")

### test after 2500 data blance

In [ ]:
augment_dataset(
    input_dataset_path='./vehicle_data/Vehicle Type Image Dataset (Version 2) VTID2 cleaned',
    output_dataset_path='./vehicle_data/Vehicle Type Image Dataset (Version 2) VTID2 balance 2500 cleaned',
    target_images_per_class=2500,
    augment_output_name="output_"
)

In [ ]:
def base_model_2500(seed=42, dataset_path='./vehicle_data/Vehicle Type Image Dataset (Version 2) VTID2 balance 2500 cleaned'):

    initialize_environment(seed)

    train_data, validation_data = prepare_data_generators(dataset_path, seed=seed)

    model = base_model(input_shape=(224, 224, 3), num_classes=5)

    history = train_model(model, train_data, validation_data, epochs=10)

    plot_training_metrics(history)

    K.clear_session()

    return model

base_model_balanced_2500 = base_model_2500(seed=42)

### test after 5000 data blance

In [ ]:
augment_dataset(
    input_dataset_path='./vehicle_data/Vehicle Type Image Dataset (Version 2) VTID2 cleaned',
    output_dataset_path='./vehicle_data/Vehicle Type Image Dataset (Version 2) VTID2 balance 5000 cleaned',
    target_images_per_class=5000,
    augment_output_name="output_"
)

In [ ]:
def base_model_5000(seed=42, dataset_path='./vehicle_data/Vehicle Type Image Dataset (Version 2) VTID2 balance 5000 cleaned'):

    initialize_environment(seed)

    train_data, validation_data = prepare_data_generators(dataset_path, seed=seed)

    model = base_model(input_shape=(224, 224, 3), num_classes=5)

    history = train_model(model, train_data, validation_data, epochs=10)

    plot_training_metrics(history)

    K.clear_session()

    return model

base_model_balanced_5000 = base_model_5000(seed=42)

## drop out test

In [ ]:
def dropout_model(input_shape=(224, 224, 3), num_classes=5, dropout_rate=0.2):
    model = Sequential()

    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
def extra_layer_model_training_pipeline(seed=42, dataset_path='./vehicle_data/Vehicle Type Image Dataset (Version 2) VTID2 balance 5000 cleaned', epochs=10, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy", model_function=None, **kwargs):

    initialize_environment(seed)

    train_data, validation_data = prepare_data_generators(dataset_path, seed=seed)

    model = model_function(input_shape=(224, 224, 3), num_classes=5, **kwargs)

    history = train_model(model, train_data, validation_data, epochs=epochs)

    plot_training_metrics(history, loss_title=loss_title, accuracy_title=accuracy_title)

    K.clear_session()

    return model, train_data, validation_data

### dropout 0.2

In [ ]:
dropout_model_1, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="0.2 Dropout - Training Loss",
    accuracy_title="0.2 Dropout - Training Accuracy",
    model_function=dropout_model,
    dropout_rate=0.2
)

### dropout 0.3

In [ ]:
dropout_model_2, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="0.3 Dropout - Training Loss",
    accuracy_title="0.3 Dropout - Training Accuracy",
    model_function=dropout_model,
    dropout_rate=0.3
)

### dropout 0.4

In [ ]:
dropout_model_3, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="0.4 Dropout - Training Loss",
    accuracy_title="0.4 Dropout - Training Accuracy",
    model_function=dropout_model,
    dropout_rate=0.4
)

## extra layer

### 3 block

In [ ]:
def extra_layer_model_3_block(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy"):
    model = Sequential()

    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
extra_layer_model_3_block, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="3 block Layer - Training Loss",
    accuracy_title="3 block Layer - Training Accuracy",
    model_function=extra_layer_model_3_block
)

### 4 block

In [ ]:
def extra_layer_model_4_block(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy"):
    model = Sequential()

    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
extra_layer_model_4_block_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="4 Block Layer - Training Loss",
    accuracy_title="4 Block Layer - Training Accuracy",
    model_function=extra_layer_model_4_block
)

## dense layer test

### 4 dense layer

In [ ]:
def extra_layer_dense_4_block(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy"):
    model = Sequential()

    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
extra_layer_dense_4_layer, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="4 dense - Training Loss",
    accuracy_title="4 dense - Training Accuracy",
    model_function=extra_layer_dense_4_block
)

### 5 dense layer

In [ ]:
def extra_layer_dense_5_block(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy"):
    model = Sequential()

    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
extra_layer_dense_instance_5_layer, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="5 dense - Training Loss",
    accuracy_title="5 dense - Training Accuracy",
    model_function=extra_layer_dense_5_block
)

In [ ]:
model = extra_layer_dense_instance_5_layer
image_paths = ["./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Hatchback\PHOTO_14.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Other\PHOTO_11.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Pickup\PHOTO_3.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Seden\PHOTO_7.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\SUV\PHOTO_17.jpg"]
lime_pipeline(model, image_paths, title="5 Dense Layer LIME Results")

## BatchNormalization

In [ ]:
def extra_layer_BatchNormalization(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy"):
    model = Sequential()

    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
extra_layer_dense_instance_BatchNormalization, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="BatchNormalization - Training Loss",
    accuracy_title="BatchNormalization - Training Accuracy",
    model_function=extra_layer_BatchNormalization
)

In [ ]:
model = extra_layer_dense_instance_BatchNormalization
image_paths = ["./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Hatchback\PHOTO_14.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Other\PHOTO_11.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Pickup\PHOTO_3.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Seden\PHOTO_7.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\SUV\PHOTO_17.jpg"]
lime_pipeline(model, image_paths, title="BatchNormalization LIME Results")

## Regularization

### L2 0.01

In [ ]:
from tensorflow.keras.regularizers import l2

In [ ]:
def extra_layer_dense_with_l2(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy"):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
extra_layer_dense_l2, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="L2 0.01 - Training Loss",
    accuracy_title="L2 0.01 - Training Accuracy",
    model_function=extra_layer_dense_with_l2
)

In [ ]:
model = extra_layer_dense_l2
image_paths = ["./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Hatchback\PHOTO_14.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Other\PHOTO_11.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Pickup\PHOTO_3.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Seden\PHOTO_7.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\SUV\PHOTO_17.jpg"]
lime_pipeline(model, image_paths, title="L2 0.01 LIME Results")

### L2 0.001

In [ ]:
def extra_layer_dense_with_l2_1(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy"):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
extra_layer_dense_l2_1, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="L2 0.001 - Training Loss",
    accuracy_title="L2 0.001 - Training Accuracy",
    model_function=extra_layer_dense_with_l2_1
)

### L2 0.0001

In [ ]:
def extra_layer_dense_with_l2_2(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy"):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.0001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
extra_layer_dense_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="L2 0.0001 - Training Loss",
    accuracy_title="L2 0.0001 - Training Accuracy",
    model_function=extra_layer_dense_with_l2_2
)

## learning rate

In [ ]:
from tensorflow.keras.optimizers import Adam

def extra_layer_learning_rate(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy", learning_rate=0.001):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

### Learning rate 0.001

In [ ]:
extra_layer_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="Learning Rate 0.001 - Training Loss",
    accuracy_title="Learning Rate 0.001 - Training Accuracy",
    model_function=extra_layer_learning_rate,
    learning_rate=0.001
)

### Learning rate 0.0001

In [ ]:
extra_layer_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="Learning Rate 0.0001 - Training Loss",
    accuracy_title="Learning Rate 0.0001 - Training Accuracy",
    model_function=extra_layer_learning_rate,
    learning_rate=0.0001
)

### Dynamic Learning rate

In [ ]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

def extra_layer_dynamic_learning_rate(input_shape=(224, 224, 3), num_classes=5, initial_learning_rate=0.0001, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy"):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    optimizer = Adam(learning_rate=initial_learning_rate)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

def train_model(model, train_data, validation_data, epochs=10):
    initial_learning_rate = 1e-4
    decay_steps = train_data.samples // train_data.batch_size * epochs
    lr_schedule = CosineDecay(initial_learning_rate, decay_steps)

    history = model.fit(
        train_data,
        validation_data=validation_data,
        epochs=epochs,
        callbacks=[reduce_lr]
    )
    return history

In [ ]:
extra_layer_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="Dynamic Learning Rate - Training Loss",
    accuracy_title="Dynamic Learning Rate - Training Accuracy",
    model_function=extra_layer_dynamic_learning_rate
)

## Optimizer

In [ ]:
from tensorflow.keras.optimizers import SGD, Nadam, Adam

In [ ]:
from tensorflow_addons.optimizers import AdamW

### SGD

In [ ]:
def SGD_model(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy"):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    optimizer = SGD(learning_rate=0.0001, momentum=0.9, nesterov=True)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
SGD_model_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="SGD Optimizer - Training Loss",
    accuracy_title="SGD Optimizer - Training Accuracy",
    model_function=SGD_model
)

### Nadam

In [ ]:
def Nadam_model(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy", learning_rate=0.001):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    optimizer = Nadam(learning_rate=0.0001)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
Nadam_model_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="Nadam Optimizer - Training Loss",
    accuracy_title="Nadam Optimizer - Training Accuracy",
    model_function=Nadam_model
)

### AdamW

In [ ]:
def AdamW_model(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy", learning_rate=0.001):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    optimizer = AdamW(learning_rate=0.0001, weight_decay=0.0001)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
AdamW_model_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="AdamW Optimizer - Training Loss",
    accuracy_title="AdamW Optimizer - Training Accuracy",
    model_function=AdamW_model
)

## Pooling method

### MaxPooling (baseline)

In [ ]:
def MaxPooling_model(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy", learning_rate=0.0001):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    optimizer = Adam(
        learning_rate=0.0001,
    )

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
MaxPooling_model_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="MaxPooling (baseline) - Training Loss",
    accuracy_title="MaxPooling (baseline) - Training Accuracy",
    model_function=MaxPooling_model
)

In [ ]:
model = MaxPooling_model_instance
image_paths = ["./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Hatchback\PHOTO_14.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Other\PHOTO_11.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Pickup\PHOTO_3.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Seden\PHOTO_7.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\SUV\PHOTO_17.jpg"]
lime_pipeline(model, image_paths, title="MaxPooling (base model) LIME Results")

### AveragePooling

In [ ]:
from tensorflow.keras.layers import AveragePooling2D

In [ ]:
def AveragePooling_model(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy", learning_rate=0.001):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    optimizer = Adam(
        learning_rate=0.0001
    )

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
AveragePooling_model_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="AveragePooling - Training Loss",
    accuracy_title="AveragePooling - Training Accuracy",
    model_function=AveragePooling_model
)

In [ ]:
model = AveragePooling_model_instance
image_paths = ["./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Hatchback\PHOTO_14.jpg"]
lime_pipeline(model, image_paths, title="AveragePooling model LIME Results")

### GlobalAveragePooling

In [ ]:
def GlobalAveragePooling_model(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy", learning_rate=0.001):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(GlobalAveragePooling2D())
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    optimizer = Adam(
        learning_rate=0.001,
    )

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
GlobalAveragePooling_model_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="GlobalAveragePooling - Training Loss",
    accuracy_title="GlobalAveragePooling - Training Accuracy",
    model_function=GlobalAveragePooling_model
)

In [ ]:
model = GlobalAveragePooling_model_instance
image_paths = ["./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Hatchback\PHOTO_14.jpg"]
lime_pipeline(model, image_paths, title="GlobalAveragePooling model LIME Results")

### GlobalMaxPooling

In [ ]:
def GlobalMaxPooling_model(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy", learning_rate=0.001):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(GlobalMaxPooling2D())
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    optimizer = Adam(
        learning_rate=0.001,
    )

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
GlobalMaxPooling_model_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="GlobalMaxPooling - Training Loss",
    accuracy_title="GlobalMaxPooling - Training Accuracy",
    model_function=GlobalMaxPooling_model
)

In [ ]:
model = GlobalMaxPooling_model_instance
image_paths = ["./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Hatchback\PHOTO_14.jpg"]
lime_pipeline(model, image_paths, title="GlobalMaxPooling model Results")

## kernel size

### size 3

In [ ]:
def kernel_size_model_1(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy", learning_rate=0.0001):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    optimizer = Adam(
        learning_rate=0.0001,
    )

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
kernel_size_model_1_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="kernel size 3 - Training Loss",
    accuracy_title="kernel size 3 - Training Accuracy",
    model_function=kernel_size_model_1
)

In [ ]:
model = kernel_size_model_1_instance
image_paths = ["./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Hatchback\PHOTO_14.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Other\PHOTO_11.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Pickup\PHOTO_3.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Seden\PHOTO_7.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\SUV\PHOTO_17.jpg"]
lime_pipeline(model, image_paths, title="kernel size 3 model Results")

### size 5

In [ ]:
def kernel_size_model_2(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy", learning_rate=0.0001):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=5, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    optimizer = Adam(
        learning_rate=0.0001,
    )

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
kernel_size_model_2_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="kernel size 5 - Training Loss",
    accuracy_title="kernel size 5 - Training Accuracy",
    model_function=kernel_size_model_2
)

In [ ]:
model = kernel_size_model_2_instance
image_paths = ["./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Hatchback\PHOTO_14.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Other\PHOTO_11.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Pickup\PHOTO_3.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Seden\PHOTO_7.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\SUV\PHOTO_17.jpg"]
lime_pipeline(model, image_paths, title="kernel size 5 model Results")

## padding method

In [ ]:
def valid_padding_model(input_shape=(224, 224, 3), num_classes=5, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy", learning_rate=0.0001):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=3, padding='valid', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=3, padding='valid', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=3, padding='valid', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=3, padding='valid', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=3, padding='valid', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=3, padding='valid', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=3, padding='valid', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=3, padding='valid', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    optimizer = Adam(
        learning_rate=0.0001,
    )

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
valid_padding_model_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="valid padding - Training Loss",
    accuracy_title="valid padding - Training Accuracy",
    model_function=valid_padding_model
)

In [ ]:
model = valid_padding_model_instance
image_paths = ["./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Hatchback\PHOTO_14.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Other\PHOTO_11.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Pickup\PHOTO_3.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\Seden\PHOTO_7.jpg", "./vehicle_data\Vehicle Type Image Dataset (Version 2) VTID2\SUV\PHOTO_17.jpg"]
lime_pipeline(model, image_paths, title="valid padding model Results")

## Final model

In [ ]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

def Final_model(input_shape=(224, 224, 3), num_classes=5, initial_learning_rate=0.0001, loss_title="Training - Loss Function", accuracy_title="Training - Accuracy"):
    model = Sequential()
    # First convolutional block
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Second convolutional block
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Third convolutional block
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Four convolutional block
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', dtype='float32'))

    # with the initial learning rate
    optimizer = Adam(learning_rate=initial_learning_rate)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
Final_model_instance, train_data, validation_data = extra_layer_model_training_pipeline(
    seed=42,
    epochs=10,
    loss_title="Dynamic Learning Rate - Training Loss",
    accuracy_title="Dynamic Learning Rate - Training Accuracy",
    model_function=Final_model
)

In [ ]:
visualkeras.layered_view(Final_model_instance, legend=True)